In [42]:
import torch
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification,Trainer
import transformers

In [43]:
transformers.__version__

'4.38.2'

In [44]:
import accelerate
accelerate.__version__

'0.27.2'

In [45]:
class newDataset:
    def __init__(self, torkenized_texts):
        self.tokenized_texts = tokenized_texts
    
    def __len__(self):
        return len(self.tokenized_texts["input_ids"])
    
    def __getitem__(self, idx):
        return {k:v[idx] for k, v in self.tokenized_texts.items()}
    
model_name = 'j-hartmann/emotion-english-distilroberta-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
trainer = Trainer(model=model)

In [46]:
file_name = 'train.csv'
text_column = 'lead_description'
df_pred = pd.read_csv(file_name)
df_pred[text_column] = df_pred[text_column].fillna('NAN')
pred_texts = df_pred[text_column].dropna().astype('str').tolist()

In [47]:
tokenized_texts = tokenizer(pred_texts, truncation=True,padding=True)
pred_texts = newDataset(tokenized_texts)

In [48]:
prediction = trainer.predict(pred_texts)

In [49]:
predictions = prediction

In [50]:
# Transform predictions to labels
preds = predictions.predictions.argmax(-1)
labels = pd.Series(preds).map(model.config.id2label)
scores = (np.exp(predictions[0])/np.exp(predictions[0]).sum(-1,keepdims=True)).max(1)

In [56]:
df.head()

,text,pred,label,score,anger,disgust,fear,joy,neutral,sadness,surprise
0,"{'input_ids': [0, 487, 1889, 2, 1, 1, 1, 1, 1,...",4,neutral,0.715612,0.024700,0.048721,0.013289,0.014079,0.715612,0.076745,0.106854
1,"{'input_ids': [0, 118, 33, 41, 793, 467, 240, ...",5,sadness,0.936021,0.011519,0.001132,0.003908,0.005819,0.014641,0.936021,0.026960
2,"{'input_ids': [0, 9112, 40455, 3842, 36759, 37...",4,neutral,0.916403,0.027019,0.005466,0.004710,0.006501,0.916403,0.018432,0.021469
3,"{'input_ids': [0, 41257, 2319, 1229, 2, 1, 1, ...",4,neutral,0.854739,0.009136,0.003865,0.005798,0.010212,0.854739,0.051155,0.065095
4,"{'input_ids': [0, 15833, 4932, 13, 275, 425, 6...",4,neutral,0.899226,0.021235,0.004996,0.006930,0.018254,0.899226,0.013417,0.035941


In [51]:
temp = (np.exp(predictions[0])/np.exp(predictions[0]).sum(-1,keepdims=True))

In [52]:
# work in progress
# container
anger = []
disgust = []
fear = []
joy = []
neutral = []
sadness = []
surprise = []

# extract scores (as many entries as exist in pred_texts)
for i in range(len(pred_texts)):
    anger.append(temp[i][0])
    disgust.append(temp[i][1])
    fear.append(temp[i][2])
    joy.append(temp[i][3])
    neutral.append(temp[i][4])
    sadness.append(temp[i][5])
    surprise.append(temp[i][6])

In [53]:
# Create DataFrame with texts, predictions, labels, and scores
df = pd.DataFrame(list(zip(pred_texts,preds,labels,scores,  anger, disgust, fear, joy, neutral, sadness, surprise)), columns=['text','pred','label','score', 'anger', 'disgust', 'fear', 'joy', 'neutral', 'sadness', 'surprise'])
df.head()

,text,pred,label,score,anger,disgust,fear,joy,neutral,sadness,surprise
0,"{'input_ids': [0, 487, 1889, 2, 1, 1, 1, 1, 1,...",4,neutral,0.715612,0.024700,0.048721,0.013289,0.014079,0.715612,0.076745,0.106854
1,"{'input_ids': [0, 118, 33, 41, 793, 467, 240, ...",5,sadness,0.936021,0.011519,0.001132,0.003908,0.005819,0.014641,0.936021,0.026960
2,"{'input_ids': [0, 9112, 40455, 3842, 36759, 37...",4,neutral,0.916403,0.027019,0.005466,0.004710,0.006501,0.916403,0.018432,0.021469
3,"{'input_ids': [0, 41257, 2319, 1229, 2, 1, 1, ...",4,neutral,0.854739,0.009136,0.003865,0.005798,0.010212,0.854739,0.051155,0.065095
4,"{'input_ids': [0, 15833, 4932, 13, 275, 425, 6...",4,neutral,0.899226,0.021235,0.004996,0.006930,0.018254,0.899226,0.013417,0.035941


In [54]:
a = df[['anger','disgust','fear','joy','neutral','sadness','surprise']]

In [57]:
a.to_csv('test_emotion.csv')

In [ ]:
# def objective(trial):
#     params = {
#         "iterations": 1000,
#         "depth": trial.suggest_int("depth", 4, 10),
#         "learning_rate": trial.suggest_float("learning_rate", 1e-3, 1.0),
#         "random_strength": trial.suggest_float("random_strength", 1e-3, 1e-1, log=True),
#         "border_count": trial.suggest_int("border_count", 1, 255),
#         "l2_leaf_reg": trial.suggest_float("l2_leaf_reg", 1e-3, 1e2, log=True),
#         "leaf_estimation_iterations": trial.suggest_int("leaf_estimation_iterations", 1, 10),
#         "leaf_estimation_method": trial.suggest_categorical("leaf_estimation_method", ["Newton", "Gradient"]),
#         "bootstrap_type": trial.suggest_categorical("bootstrap_type", ["Bayesian", "Bernoulli", "MVS"]),
#         "grow_policy": trial.suggest_categorical("grow_policy", ["SymmetricTree", "Depthwise", "Lossguide"]),
#         "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 1, 100),
#         "one_hot_max_size": 1,
#         "random_state": seed,
#         "verbose": 0,
#     }
    
#     #score = []
#     tmp_score = []
#     for train_pool, val_pool, y_true in zip(all_train_pool, all_val_pool, all_y_val):
#         clf = CatBoostClassifier(**params,
#                                  scale_pos_weight=np.sqrt(rate), 
#                                  loss_function='Logloss',
#                                  eval_metric='F1',
#                                  early_stopping_rounds=100,
#                                  #task_type="GPU",
#                                 )
#         clf.fit(train_pool, eval_set=val_pool)
# #         y_pred = clf.predict(val_pool)
# #         y_pred = [False if i==0 else True for i in y_pred]
# #         tmp_score.append(f1_score(y_true,y_pred,labels=[True, False]))
#         tmp_score.append(clf.get_feature_importance()[5])
#     tmp_score = np.mean(tmp_score)
#     return tmp_score

# # Hyperparameter Tuning
# study = optuna.create_study(direction='maximize', sampler=TPESampler(seed=seed), pruner=SuccessiveHalvingPruner())
# study.optimize(objective, n_trials=None)